# Notebook 10 - Generate Store-Level Meal Boxes

This notebook integrates product availability and waste-aware prioritization to select **fully matched recipe plans per store**. It ensures that:

- All required ingredients for a recipe are available in the same store.
- At least one ingredient is flagged as **wasted or markdown**, based on recent snapshot data.
- Recipes are ranked using the average priority score of the ingredients they require.

Inputs:
- `products_with_priority.csv`: Product metadata with waste/markdown flags
- `store_recipe_matches.csv`: Ingredient–store matching results
- `recipes_with_ontology.csv`: Recipe–ingredient–concept mappings

Output:
- `store_mealbox_candidates.csv`: Ranked recipe candidates per store with complete and relevant ingredient matches


In [11]:
import pandas as pd
import os

# Paths
input_folder = "cleaned_data"
output_folder = "cleaned_data"

# Load datasets
df_products = pd.read_csv(os.path.join(input_folder, "products_with_priority.csv"))
df_matches = pd.read_csv(os.path.join(input_folder, "store_recipe_matches.csv"))
df_recipes = pd.read_csv(os.path.join(input_folder, "recipes_with_ontology.csv"))

# Patch: Add recipe column to df_matches if missing
if "recipe" not in df_matches.columns:
    df_matches = df_matches.merge(df_recipes[["ingredient", "recipe"]], on="ingredient", how="left")

# Filter to ingredients available in store
df_available = df_matches[df_matches["match_available"] == True].copy()
print("Available ingredient-store matches:", df_available.shape)


C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,3,5,31) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Available ingredient-store matches: (9, 5)


In [12]:
# Step 1 – Compute which recipes are fully matched
ingredients_per_recipe = df_matches.groupby("recipe")["ingredient"].nunique().to_dict()
grouped = df_available.groupby(["store", "recipe"])["ingredient"].nunique().reset_index()
grouped["total_required"] = grouped["recipe"].map(ingredients_per_recipe)
grouped["complete"] = grouped["ingredient"] == grouped["total_required"]
df_complete = grouped[grouped["complete"]].copy()

# Step 2 – Merge ingredients + priority info
df_products_subset = df_products[["store", "article", "product_concept", "priority_score", "waste_flag", "markdown_flag"]].dropna()
df_enriched = df_complete.merge(df_matches, on=["store", "recipe"], how="left")

df_enriched = df_enriched.merge(
    df_products_subset,
    left_on=["store", "ingredient_concept"],
    right_on=["store", "product_concept"],
    how="left"
)

# Step 3 – Keep only recipes with at least one ingredient flagged
df_enriched["flagged"] = df_enriched[["waste_flag", "markdown_flag"]].sum(axis=1)
df_enriched = df_enriched[df_enriched["flagged"] > 0]
print("Complete and concept-flagged matches:", df_enriched.shape)


Complete and concept-flagged matches: (10, 14)


In [13]:
# Compute average priority per meal box (store–recipe)
box_priority = (
    df_enriched.groupby(["store", "recipe"])
    .agg(avg_priority=("priority_score", "mean"))
    .reset_index()
)

# Sort for top-ranked recipes per store
box_priority = box_priority.sort_values(["store", "avg_priority"], ascending=[True, False])
display(box_priority.head())

# Save output
output_path = os.path.join(output_folder, "store_mealbox_candidates.csv")
box_priority.to_csv(output_path, index=False)
print("-> Saved store meal box candidates to:", output_path)


,store,recipe,avg_priority
0,1024,Greek Yogurt & Honey,1
1,1058,Greek Yogurt & Honey,1
2,1160,Greek Yogurt & Honey,1
3,3298,Greek Yogurt & Honey,1
4,3345,Greek Yogurt & Honey,1


-> Saved store meal box candidates to: cleaned_data\store_mealbox_candidates.csv


### Summary

This notebook implements a robust meal box planning step by:

- Identifying which recipes can be fully constructed from available store inventory.
- Filtering out recipes that don't use any waste- or markdown-prioritized products.
- Ranking remaining recipe-store pairs by average **priority_score**, enabling targeted interventions at high-waste locations.

This strategy ensures both **completeness** and **waste-reduction relevance**, aligning with the project goal of minimizing retail and consumer food waste using intelligent meal planning.

-> Output written to: `cleaned_data/store_mealbox_candidates.csv`

